In [ ]:
from pathlib import Path
import xarray as xr
from utils import model_info, general
import scipy

: 

In [ ]:
dpath_str = "/N/slate/jmelms/projects/dcmip2025_idealized_tests/experiments/H.energy_balance/data/E3SM_runs/v2.1.WCYCLSSP370_20180201-20180401_12n32p_original_20251013/run/v2.1.WCYCLSSP370_20180201-20180401_12n32p_original_20251013.eam.h1.2018-02-01-00000.nc"
dpath = Path(dpath_str)
print("Retrieving data")

In [ ]:
full_ds = xr.open_dataset(dpath)
full_ds

In [ ]:
vars_in_ds = (set(map(str.upper, model_info.MASTER_VARIABLES_NAMES)) & set(full_ds.data_vars)) | {"TMQ", "PS", "PHIS", "U050", "V050", "T050", "Z050", "Q050", "lat"}
e3sm_to_cds = {var: var.lower() for var in vars_in_ds}
e3sm_to_cds["U050"] = "u50" # 50 hPa zonal wind
e3sm_to_cds["V050"] = "v50" # 50 hPa meridional wind
e3sm_to_cds["T050"] = "t50" # 50 hPa temperature
e3sm_to_cds["Z050"] = "z50" # 50 hPa geopotential height
e3sm_to_cds["Q050"] = "q50" # 50 hPa specific humidity
e3sm_to_cds["PS"] = "sp" # surface pressure
e3sm_to_cds["PHIS"] = "z" # surface geopotential
e3sm_to_cds["TMQ"] = "tcwv" # total column water vapor
ds = full_ds[vars_in_ds].rename(e3sm_to_cds)
ds

In [ ]:
for var in full_ds.data_vars:
    if var not in vars_in_ds:
        print(f"{var}: {full_ds[var].attrs.get('long_name', '')}")

In [ ]:
# Set constants
cp = 1005.0  # J/kg/K
g = 9.81  # m/s^2
Lv = 2.26e6  # J/kg
sb_const = 5.670374419e-8  # W/m^2/K^4, from https://physics.nist.gov/cgi-bin/cuu/Value?sigma

In [ ]:
## Step 4a: Compute a mask of the p-levels above the surface for integration ###
sp = ds["sp"] / 100
sp_expanded = sp.expand_dims(dim={"level": ds.sizes["level"]}, axis=-1)
expand_dict = {dim: ds.sizes[dim] for dim in ds.dims if dim != "level"}
levs_expanded = ds["level"].expand_dims(dim=expand_dict)
mask = levs_expanded <= sp_expanded
ds["terrain_mask"] = mask

### Step 4b: Get pressure for integration ###
pa = 100 * ds.level.values # convert to Pa from hPa, used for integration

### Step 4c: Calculate total energy components ###
# sensible heat
sensible_heat = cp * ds["T"]
# latent heat - this is already column-integrated
latent_heat = Lv * ds["TCW"]
# geopotential energy
geopotential_energy = ds["Z"] # geopotential energy is already in J/kg, no need to multiply by g
# kinetic energy
kinetic_energy = 0.5 * ds["U"] ** 2 + 0.5 * ds["V"] ** 2

### Step 4d: Calculate total energy by adding all components ###
# total energy minus latent heat
dry_total_energy = sensible_heat + geopotential_energy + kinetic_energy
dry_total_energy = dry_total_energy.where(mask, 0.0) # set values below surface to 0
# column integration
print(f"Integrating dry total energy with shape {dry_total_energy.shape} and pa with shape {pa.shape}")
dry_total_energy_column = (1 / g) * scipy.integrate.trapezoid( # add nan-safe version
    dry_total_energy, pa, axis=3
)

# sum
ds["VAR_TE"] = (expand_dict, dry_total_energy_column + latent_heat.values)
ds["VAR_TE"] = ds["VAR_TE"].assign_attrs(
    {"units": "J/m^2", "long_name": "Total Energy"}
)

### Step 4e: Weight by latitude ####
# get latitude weighted total energy (time, ensemble)
ds["LW_TE"] = general.latitude_weighted_mean(ds["VAR_TE"], ds.latitude)
ds["LW_TE"].assign_attrs(
    {"units": "J/m^2", "long_name": "Latitude-Weighted Total Energy"}
)